# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [8]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
from pathlib import Path
import re
import numpy as np
import json

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import words

In [9]:
# load data from database
repo = Path.cwd()
path = repo / 'data' / 'DisasterResponse.db'

engine = create_engine(f'sqlite:///{path}')
df = pd.read_sql("select * from Message", con=engine)
df.shape

(25991, 38)

Check imbalance in target. See explore_data.ipynb

### 2. Write a tokenization function to process your text data

In [11]:
stop_words = stopwords.words("english")
vocab = set(words.words())
vocab = vocab - set(stop_words)
lemmatizer = WordNetLemmatizer()
words_set = {lemmatizer.lemmatize(word) for word in vocab}

In [12]:
def tokenize(text):
    """TO DO
    """
    
    text = re.sub(r"[^a-zA-Z]", " ", text.lower())
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = [lemmatizer.lemmatize(token).strip() for token in tokens if token in words_set]
    
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [413]:
pipeline = Pipeline([
    ("cv", CountVectorizer(tokenizer=tokenize)),
    ("transformer", TfidfTransformer()),
    ("dtc", MultiOutputClassifier(DecisionTreeClassifier(random_state=42, class_weight="balanced"), n_jobs=-1))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [414]:
X = df['message'].values
Y = df.iloc[:, :-3].values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((20792,), (5199,), (20792, 35), (5199, 35))

In [415]:
pipeline.fit(X_train, Y_train)

Pipeline(steps=[('cv',
                 CountVectorizer(tokenizer=<function tokenize at 0x12fb5fe50>)),
                ('transformer', TfidfTransformer()),
                ('dtc',
                 MultiOutputClassifier(estimator=DecisionTreeClassifier(class_weight='balanced',
                                                                        random_state=42),
                                       n_jobs=-1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [416]:
Y_preds = pipeline.predict(X_test)

In [417]:
# Y_probas = pipeline.predict_proba(X_test) # used to test different threshold values
# Y_probas_1 = np.array([prob_mat[:,1] for prob_mat in Y_probas])

In [418]:
# from sklearn.metrics import confusion_matrix
# from sklearn.metrics import precision_score
# from sklearn.metrics import recall_score

In [419]:
# # th = {0.2, 0.4, 0.6, 0.8}
# th = 0.2
# for idx, _ in enumerate(output_labels):
#     Y_pred_th = np.transpose(np.apply_along_axis(lambda x: (x > th).astype(int), 1, Y_probas_1))
#     #print(confusion_matrix(Y_test[:, idx], Y_pred_th[:, idx]))
#     print(precision_score(Y_test[:, idx], Y_pred_th[:, idx]))
#     print(recall_score(Y_test[:, idx], Y_pred_th[:, idx]))
#     print("------------------------------------")

In [420]:
def get_metrics(y_test, y_pred):
    """TO DO
    """
    cr = classification_report(y_test, y_pred, zero_division=0, output_dict=True)
    return cr

In [421]:
report_dict = {}
output_labels = list(df.iloc[:, :-3].columns)
# num_outputs = len(output_labels)
# num_outputs

In [422]:
import numpy as np
for idx, output in enumerate(output_labels):
    metrics_dict = get_metrics(Y_test[:,idx], Y_preds[:,idx])
    report_dict[output] = metrics_dict

In [423]:
with open('dtc_model_results.json', 'w', encoding="utf-8") as fhand:
    json.dump(report_dict, fhand)

In [424]:
for k, v in report_dict.items():
    print(f"""category: {k} \n accuracy: {v['accuracy']:.4f}, precision: {v['1']['precision']:.4f}, recall: {v['1']['recall']:.4f}, f-score: {v['1']['f1-score']:.4f}""")
    

category: related 
 accuracy: 0.7711, precision: 0.8604, recall: 0.8380, f-score: 0.8490
category: request 
 accuracy: 0.8215, precision: 0.5000, recall: 0.5830, f-score: 0.5383
category: offer 
 accuracy: 0.9740, precision: 0.0088, recall: 0.0417, f-score: 0.0146
category: aid_related 
 accuracy: 0.6905, precision: 0.6278, recall: 0.6425, f-score: 0.6351
category: medical_help 
 accuracy: 0.8704, precision: 0.3096, recall: 0.4212, f-score: 0.3569
category: medical_products 
 accuracy: 0.9033, precision: 0.2019, recall: 0.3537, f-score: 0.2570
category: search_and_rescue 
 accuracy: 0.9236, precision: 0.1040, recall: 0.2464, f-score: 0.1462
category: security 
 accuracy: 0.9375, precision: 0.0558, recall: 0.1744, f-score: 0.0845
category: military 
 accuracy: 0.9379, precision: 0.2459, recall: 0.4464, f-score: 0.3171
category: water 
 accuracy: 0.9473, precision: 0.5471, recall: 0.6913, f-score: 0.6108
category: food 
 accuracy: 0.9292, precision: 0.6794, recall: 0.7184, f-score: 0.698

### 6. Improve your model
Use grid search to find better parameters. 

In [354]:
pipeline.get_params()

{'memory': None,
 'steps': [('cv',
   CountVectorizer(tokenizer=<function tokenize at 0x12fb5fe50>)),
  ('transformer', TfidfTransformer()),
  ('dtc',
   MultiOutputClassifier(estimator=DecisionTreeClassifier(), n_jobs=-1))],
 'verbose': False,
 'cv': CountVectorizer(tokenizer=<function tokenize at 0x12fb5fe50>),
 'transformer': TfidfTransformer(),
 'dtc': MultiOutputClassifier(estimator=DecisionTreeClassifier(), n_jobs=-1),
 'cv__analyzer': 'word',
 'cv__binary': False,
 'cv__decode_error': 'strict',
 'cv__dtype': numpy.int64,
 'cv__encoding': 'utf-8',
 'cv__input': 'content',
 'cv__lowercase': True,
 'cv__max_df': 1.0,
 'cv__max_features': None,
 'cv__min_df': 1,
 'cv__ngram_range': (1, 1),
 'cv__preprocessor': None,
 'cv__stop_words': None,
 'cv__strip_accents': None,
 'cv__token_pattern': '(?u)\\b\\w\\w+\\b',
 'cv__tokenizer': <function __main__.tokenize(text)>,
 'cv__vocabulary': None,
 'transformer__norm': 'l2',
 'transformer__smooth_idf': True,
 'transformer__sublinear_tf': Fals

In [368]:
from sklearn.model_selection import GridSearchCV

param_grid = {
              "cv__max_df": (0.5, 1.0),  
              "dtc__estimator__min_samples_leaf": [5, 10],
              }

cv = GridSearchCV(pipeline, param_grid=param_grid, scoring="f1_micro")

In [369]:
cv.fit(X_train, Y_train)

GridSearchCV(estimator=Pipeline(steps=[('cv',
                                        CountVectorizer(tokenizer=<function tokenize at 0x12fb5fe50>)),
                                       ('transformer', TfidfTransformer()),
                                       ('dtc',
                                        MultiOutputClassifier(estimator=DecisionTreeClassifier(class_weight='balanced',
                                                                                               random_state=42),
                                                              n_jobs=-1))]),
             param_grid={'cv__max_df': (0.5, 1.0),
                         'dtc__estimator__min_samples_leaf': [5, 10]},
             scoring='f1_micro')

In [371]:
cv.best_estimator_

Pipeline(steps=[('cv',
                 CountVectorizer(max_df=0.5,
                                 tokenizer=<function tokenize at 0x12fb5fe50>)),
                ('transformer', TfidfTransformer()),
                ('dtc',
                 MultiOutputClassifier(estimator=DecisionTreeClassifier(class_weight='balanced',
                                                                        min_samples_leaf=5,
                                                                        random_state=42),
                                       n_jobs=-1))])

In [372]:
cv.best_params_

{'cv__max_df': 0.5, 'dtc__estimator__min_samples_leaf': 5}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [408]:
Y_preds = cv.predict(X_test)

In [409]:
Y_preds.shape

(5199, 35)

In [410]:
for idx, output in enumerate(output_labels):
    metrics_dict = get_metrics(Y_test[:,idx], Y_preds[:,idx])
    report_dict[output] = metrics_dict

In [411]:
with open('dtc_model_gridsearchcv_results.json', 'w', encoding="utf-8") as fhand:
    json.dump(report_dict, fhand)

In [412]:
for k, v in report_dict.items():
    print(f"""category: {k} \n accuracy: {v['accuracy']:.4f}, precision: {v['1']['precision']:.4f}, recall: {v['1']['recall']:.4f}, f-score: {v['1']['f1-score']:.4f}""")

category: related 
 accuracy: 0.7305, precision: 0.8720, recall: 0.7608, f-score: 0.8126
category: request 
 accuracy: 0.7725, precision: 0.4192, recall: 0.7123, f-score: 0.5277
category: offer 
 accuracy: 0.9783, precision: 0.0110, recall: 0.0417, f-score: 0.0174
category: aid_related 
 accuracy: 0.6851, precision: 0.6219, recall: 0.6347, f-score: 0.6282
category: medical_help 
 accuracy: 0.8036, precision: 0.2281, recall: 0.5450, f-score: 0.3216
category: medical_products 
 accuracy: 0.8604, precision: 0.1450, recall: 0.3984, f-score: 0.2126
category: search_and_rescue 
 accuracy: 0.9019, precision: 0.0885, recall: 0.2899, f-score: 0.1356
category: security 
 accuracy: 0.9154, precision: 0.0366, recall: 0.1628, f-score: 0.0598
category: military 
 accuracy: 0.9144, precision: 0.1874, recall: 0.4940, f-score: 0.2717
category: water 
 accuracy: 0.9248, precision: 0.4333, recall: 0.8360, f-score: 0.5708
category: food 
 accuracy: 0.9025, precision: 0.5475, recall: 0.8364, f-score: 0.661

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [425]:
import pickle

pickle.dump(cv, open('model_dtc_gridsearchcv.pkl','wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.